# API Search - Make our bot to talk to any API

We have observed the remarkable synergy created by combining **GPT-4 with intelligent agents and detailed prompts**. This powerful combination has consistently delivered impressive results. To further capitalize on this capability, we should aim to integrate it with various systems through API communication. Essentially, we can develop within this notebook what is referred to in OpenAI's ChatGPT as 'GPTs.'

Envision a bot that seamlessly integrates with:

- **CRM Systems:** Including Dynamics, Salesforce, and HubSpot.
- **ERP Systems:** Such as SAP, Dynamics, and Oracle.
- **CMS Systems:** Including Adobe, Oracle, and other content management platforms.

The objective is to connect our bot with data repositories, minimizing data duplication as much as possible. These systems typically offer APIs, facilitating programmatic data access.

In this notebook, we plan to develop an agent capable of querying an API to retrieve information and effectively answer questions. We will maintain our focus on the COVID-19 theme and interact with the Open Disease Data API (https://disease.sh/).

In [1]:
import json
import requests
from time import sleep
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.requests import RequestsWrapper
from langchain.chains import APIChain
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

from common.callbacks import StdOutCallbackHandler
from common.utils import num_tokens_from_string, reduce_openapi_spec
from common.prompts import APISEARCH_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))


In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

llm_1 = AzureChatOpenAI(deployment_name="gpt-4-32k", temperature=0, max_tokens=2000, callback_manager=cb_manager)
llm_2 = AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", temperature=0, max_tokens=1000)


## The Logic

By now, you must infer that the solution for an API Agent has to be something like: give the API specification as part of the system prompt to the LLM , then have an agent plan for the right steps to formulate the API call.<br>

Let's do that. But we must first understand the industry standards of Swagger/OpenAPI


## Introduction to OpenAPI (formerly Swagger)

The OpenAPI Specification, previously known as the Swagger Specification, is a specification for a machine-readable interface definition language for describing, producing, consuming and visualizing web services. Previously part of the Swagger framework, it became a separate project in 2016, overseen by the OpenAPI Initiative, an open-source collaboration project of the Linux Foundation.

OpenAPI Specification is an API description format for REST APIs. An OpenAPI file allows you to describe your entire API, including: Available endpoints (/users for example) and operations on each endpoint ( GET /users, POST /users), description, contact information, license, terms of use and other information.

### Let's get the OpenAPI (Swagger) spec from our desired API that we want to talk to

In [6]:
url = 'https://disease.sh/apidocs/swagger_v3.json'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    spec = response.json()
else:
    spec = None
    print(f"Failed to retrieve data: Status code {response.status_code}")


Let's see how big is this API specification:

In [7]:
# You can check the function "reduce_openapi_spec()" in utils.py
reduced_api_spec = reduce_openapi_spec(spec)

In [8]:
api_tokens = num_tokens_from_string(str(spec))
print("API spec size in tokens:",api_tokens)
api_tokens = num_tokens_from_string(str(reduced_api_spec))
print("Reduced API spec size in tokens:",api_tokens)

API spec size in tokens: 12855
Reduced API spec size in tokens: 8772


Sometimes it makes sense to reduce the size of the API Specs by using the `reduce_openapi_spec` function. It's optional.

## Question
Let's make a complicated question that requires two distinct API calls to different endpoints:

In [9]:
QUESTION = "What is to-date the amount of people tested in Argentina vs USA? Also tell me what is the continent with more covid deaths and what's the count as of today"

## Use a chain to convert the natural language question to an API request using the API specification in the prompt

We can use a nice chain in langchain called APIChain

In [10]:
# Most of APIs require Authorization tokens, so we construct the headers using a lightweight python request wrapper called RequestsWrapper
access_token = "ABCDEFG123456" 
headers = {"Authorization": f"Bearer {access_token}"}
requests_wrapper = RequestsWrapper(headers=headers)

**Note**: Notice that we are using GPT-3.5 (llm_2) below for this chain since it doesn't need too many instructions or reasoning

In [11]:
chain = APIChain.from_llm_and_api_docs(
    llm=llm_2,
    api_docs=str(reduced_api_spec),
    headers=headers,
    verbose=True,
    limit_to_domains=["https://disease.sh/"],
)


These are the prompts on the APIChain class (on to create the URL endpoint and the other one to use it and get the answer):

In [12]:
chain.api_request_chain.prompt.template

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url:'

In [13]:
chain.api_answer_chain.prompt.template

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url: {api_url}\n\nHere is the response from the API:\n\n{api_response}\n\nSummarize this response to answer the original question.\n\nSummary:'

In [14]:
try:
    printmd(chain.run(QUESTION))
except Exception as e:
    print(e)



> Entering new APIChain chain...
https://disease.sh/v3/covid-19/countries/Argentina,USA
[{"updated":1702079763401,"country":"Argentina","countryInfo":{"_id":32,"iso2":"AR","iso3":"ARG","lat":-34,"long":-64,"flag":"https://disease.sh/assets/img/flags/ar.png"},"cases":10080046,"todayCases":0,"deaths":130685,"todayDeaths":0,"recovered":9949361,"todayRecovered":0,"active":0,"critical":0,"casesPerOneMillion":219083,"deathsPerOneMillion":2840,"tests":35716069,"testsPerOneMillion":776264,"population":46010234,"continent":"South America","oneCasePerPeople":5,"oneDeathPerPeople":352,"oneTestPerPeople":1,"activePerOneMillion":0,"recoveredPerOneMillion":216242.35,"criticalPerOneMillion":0},{"updated":1702079763372,"country":"USA","countryInfo":{"_id":840,"iso2":"US","iso3":"USA","lat":38,"long":-97,"flag":"https://disease.sh/assets/img/flags/us.png"},"cases":109719842,"todayCases":4440,"deaths":1184570,"todayDeaths":11,"recovered":107579405,"todayRecovered":2666,"active":955867,"critical":1479,

To-date, the number of people tested in Argentina is 35,716,069, while in the USA it is 1,186,431,916. The continent with the highest number of COVID deaths is North America, with a total count of 1,184,570 deaths as of today.

As we have seen before in prior notebooks, a single chain cannot reason/observe/think, so it cannot figure out that it needs to call two endpoints in order to get the continents information. As you will see below, North America is NOT the continent with the highest amount of deaths.

## Creating a custom agent that uses the APIChain as a tool

To solve the avobe problem, we can build a REACT Agent that uses the APIChain as a tool to get the information. This agent will create as many calls as needed (using the chain tool) until it answers the question

In [15]:
class MyAPISearch(BaseTool):
    """APIChain as an agent tool"""
    
    name = "@apisearch"
    description = "useful when the questions includes the term: @apisearch.\n"

    llm: AzureChatOpenAI
    api_spec: str
    headers: dict = {}
    limit_to_domains: list = []
    verbose: bool = False
    
    def _run(self, query: str) -> str:
        
        chain = APIChain.from_llm_and_api_docs(
                            llm=self.llm,
                            api_docs=self.api_spec,
                            headers=self.headers,
                            verbose=self.verbose,
                            limit_to_domains=self.limit_to_domains
                            )
        try:
            sleep(2) # This is optional to avoid possible TPM rate limits
            response = chain.run(query)
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

Notice below that we are using GPT-35-Turbo-16k (llm_2) for the Tool and GPT-4-turbo (llm_1) for the Agent

In [16]:
tools = [MyAPISearch(llm=llm_2, api_spec=str(spec), limit_to_domains=["https://disease.sh/"])]
agent_executor = initialize_agent(tools, llm_1, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, agent_kwargs={'prefix':APISEARCH_PROMPT_PREFIX}, callback_manager=cb_manager)

In [17]:
# Uncomment if you want to see the prompt
# printmd(agent_executor.agent.llm_chain.prompt.template)

In [18]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue
        
printmd(response)

The user is asking for three pieces of information: 
1. The number of people tested for COVID-19 in Argentina and the USA to date.
2. The continent with the highest number of COVID-19 deaths.
3. The count of COVID-19 deaths on that continent as of today.

I will need to use my search tool to find this information. I'll start by searching for the number of people tested for COVID-19 in Argentina and the USA. Then, I'll search for the continent with the highest number of COVID-19 deaths and the current count. 

Action: @apisearch
Action Input: Number of people tested for COVID-19 in Argentina and USA to date
I have found the number of people tested for COVID-19 in Argentina and the USA. Now, I will search for the continent with the highest number of COVID-19 deaths and the current count.

Action: @apisearch
Action Input: Continent with the highest number of COVID-19 deaths and current count


As of the latest update, Argentina has conducted 35,716,069 COVID-19 tests, while the USA has conducted 1,186,431,916 tests. The continent with the highest number of COVID-19 deaths is Europe, with a current count of 2,086,401 deaths.

**Great!!** we have now an API Agent using APIChain as a tool, capable of reasoning until it can find the answer. And it is pretty fast as well.

## Simple APIs

What happens if the API is quite basic, meaning it's just a simple endpoint without a Swagger/OpenAPI definition? Let’s consider the following example:

[CountdownAPI](https://www.countdownapi.com/) is a streamlined version of the eBay API, available as a paid service. We can test it using their demo query, which does not require any Swagger or OpenAPI specification. In this scenario, our main task is to create a tool that retrieves the results. We then pass these results to an agent for analysis, providing answers to user queries, similar to our approach with the Bing Search agent.

An aspect we haven't discussed yet while constructing our API Agent using the APIChain tool is handling situations where either the API specification or the API call results are quite extensive. In such cases, we need to choose between using GPT-4-32k and GPT-4-Turbo.

In the example below, there is no API specification, but the response from the API is rather lengthy. For this scenario, we will employ GPT-4-32k.

In [19]:
# set up the request parameters
params = {
  'api_key': 'demo',
  'type': 'search',
  'ebay_domain': 'ebay.com',
  'search_term': 'memory cards'
}

# make the http GET request to Countdown API
api_result = requests.get('https://api.countdownapi.com/request', params)

num_tokens = num_tokens_from_string(str(api_result.json())) # this is a custom function we created in common/utils.py
print("Token count:",num_tokens,"\n")  

# print the first 2000 characters of JSON response from Countdown API
print(json.dumps(api_result.json())[:2000], "...")

Token count: 16230 

{"request_info": {"success": true, "demo": true}, "request_parameters": {"type": "search", "ebay_domain": "ebay.com", "search_term": "memory cards"}, "request_metadata": {"ebay_url": "https://www.ebay.com/sch/i.html?_nkw=memory+cards&_sacat=0&_dmd=1&_fcid=1"}, "search_results": [{"position": 1, "title": "Sandisk Micro SD Card Memory 32GB 64GB 128GB 256GB 512GB 1TB Lot Extreme Ultra", "epid": "203914554350", "link": "https://www.ebay.com/itm/203914554350", "image": "https://i.ebayimg.com/thumbs/images/g/A7wAAOSwemNjTz~l/s-l300.jpg", "condition": "Brand New", "seller_info": {"name": "terashack", "review_count": 58982, "positive_feedback_percent": 100}, "is_auction": false, "buy_it_now": false, "free_returns": true, "sponsored": true, "prices": [{"value": 9.99, "raw": "$9.99"}, {"value": 438.99, "raw": "$438.99"}], "price": {"value": 9.99, "raw": "$9.99"}}, {"position": 2, "title": "SanDisk 512GB Extreme PRO CFexpress Memory Card Type B - SDCFE-512G-ANCIN", "epid": "2

So, the answer from this product query (the demo only works with 'memory cards' - you will need to sign up for their trial if you want to try any query with an API key), is about 16.5k tokens. When combined with the prompt, we won't have any other option than to use GPT-4-32k or GPT-4 turbo models. 

In [20]:
class MySimpleAPISearch(BaseTool):
    """Tool for simple API calls that doesn't require OpenAPI 3.0 specs"""
    
    name = "@apisearch"
    description = "useful when the questions includes the term: @apisearch.\n"

    api_key: str
    
    def _run(self, query: str) -> str:
        
        params = {
          'api_key': self.api_key,
          'type': 'search',
          'ebay_domain': 'ebay.com',
          'search_term': query
        }

        # make the http GET request to Countdown API
        api_result = requests.get('https://api.countdownapi.com/request', params)
        
        try:
            response = json.dumps(api_result.json())
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

In [21]:
tools = [MySimpleAPISearch(api_key='demo')]
agent_executor = initialize_agent(tools, llm_1, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, agent_kwargs={'prefix':APISEARCH_PROMPT_PREFIX}, callback_manager=cb_manager)

In [22]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run('what is the price for SanDisk memory cards? give me the links please') 
        break
    except Exception as e:
        response = str(e)
        continue

printmd(response)

The user is asking for the price of SanDisk memory cards and also wants the links to the sources of this information. I will use the @apisearch tool to find this information.
Action: @apisearch
Action Input: SanDisk memory card price
The search did not return any results. I should try again with slightly different search terms.
Action: @apisearch
Action Input: price of SanDisk memory cards
The user is asking for the price of SanDisk memory cards and also wants the links to the sources of this information. I will use the @apisearch tool to find this information.
Action: @apisearch
Action Input: SanDisk memory card price
The search was unsuccessful due to an issue with the search tool. I will attempt the search again with slightly modified search terms.
Action: @apisearch
Action Input: price of SanDisk memory cards


An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The search was successful and I found several listings for SanDisk memory cards on eBay. The prices vary depending on the capacity and condition of the memory cards. Here are some examples:

1. [SanDisk High Endurance Micro SD Memory Card 32GB 64GB 128GB 256GB V30 C10 CCTV](https://www.ebay.com/itm/204531908628) - Price ranges from USD 16.99 to USD 308.99.
2. [Sandisk SD Cards 16GB 32GB 64GB 128GB 256GB Extreme Pro Ultra Memory Cards lot](https://www.ebay.com/itm/324078167020) - Price ranges from USD 7.98 to USD 236.74.
3. [Lot Qty 4 of SanDisk 32GB Flash SD Memory Camera Card C4](https://www.ebay.com/itm/372019364336) - Price is USD 23.98.
4. [Sandisk Ulta Value Pack 5 SD Cards Includes 2x4GB,2x8GB, 1x16GB](https://www.ebay.com/itm/155937428613) - Price is USD 20.
5. [SanDisk offers 32GB, 64GB, and 128GB microSD cards for various storage needs](https://www.ebay.com/itm/404532089555) - Price ranges from USD 9.99 to USD 17.77.

Please note that these prices are subject to change and may not include shipping costs. It's also important to check the condition of the item and the seller's feedback before making a purchase.`

# Summary

In this notebook, we learned about how to create very smart API agents for simple or complex APIs that use Swagger or OpenAPI specifications.
We see, again, that the key to success is to use: Agents with Expert tools + GPT-4 + good prompts.

As homework, try to create a shopping assistant for Etsy e-commerce site using the following API spec: (you will need to register for free and create an API-Key)

- https://developers.etsy.com/documentation/
- https://www.etsy.com/openapi/generated/oas/3.0.0.json

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.